# Task 02: Solar Dish with Stirling Engine

(Author: Rolf Becker, Date: 2022-07-21, License: CC-BY-SA-4.0)

## Introduction

**Concentrated Solar Power**
<br>
The principle of concentrated solar power (CSP) is used in solar thermal power plants. Incoming irradiance is concentrated by mirrors and focused on a thermal collector, where a medium is heated. The most common approach is to use parabolic troughs heating up collector pipes in the focal line of the troughs. The collector pipes are filled with a heat transfer fluid transporting the thermal energy to a heat engine (e.g. a steam turbine) connected to an electrical generator. 

Another way - for several reasons less common - is the use of a parabolic mirror (aka “dish”) focusing on the hot end of a Stirling engine. 
<br>**Your tasks:**

1. Analyse a specified Stirling engine.
1. Find an appropriate location with suficient mean irradiance in your home country.
1. Dimension an appropriate parabolic mirror to collect the necessary thermal input power.

The hot end of the engine is installed in the focus of this solar concentrator. The system is mounted on a solar tracker such that a direct sun ray is normal to the center of the dish. Thus the direct normal irradiance and partly the circumsolar diffuse radiation are the major irradiance components for the system.


<img src="./images/600px-Dish-stirling-at-odeillo.jpg" width="400">

_Fig.1: 10 kW Dish-Stirling system in Font-Romeu-Odeillo, France._
<br>(Author: Björn Appel, Date: 2005-09-20, License: CC-BY-SA-2.5, 
<br>Source: https://de.wikipedia.org/wiki/Solar-Stirling)


## 0. Preparation

**Q 0.1:** A similar task for a Physics 2 project report was set in 2021. You find further information on [**teawiki**](https://teawiki.net/doku.php?id=solar_dish:start).<br>
**Read it!** It provides some **more background information on Stirling engines** mounted to solar dishes. 

**Assumptions / Definitions:**

* We use the following form of the 1st law of thermodynaics: $\Delta U = Q + W$
* The internal energy $U$ can be increased ($\Delta U$ positive) by adding heat $Q$ to or doing work $W$ on the system.  
* Heating up means positive $Q$ (addition of heat), cooling down means negative $Q$ (extraction of heat).
* The work done on the system is counted as positive: $W > 0$. The work done by the system is counted negative: $W < 0$.
* The ideal gas law is the state equation describing the system sufficiently well: $pV = nRT$
* The state transitions between the states (therodynamic processes) are quasi-static.
* We assume an ideal Stirling engine.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.constants import R
print(f"Molar Gas Constant: {R = :.3f} J/(K*mol)")

Molar Gas Constant: R = 8.314 J/(K*mol)


### Construction of Needed Data Structures to be Filled

Your task is to **fill the state and process tables completely!** All calculations have to be **based on these tables!**  

In [2]:
# Create pandas dataframe for the thermodynamic states to be filled. Values initialized with 0.
states = pd.DataFrame(
    {
        "ID" : ["A", "B", "C", "D"] , 
        "n/mol" : np.zeros(4) , 
        "p/Pa"  : np.zeros(4) , 
        "V/m3"  : np.zeros(4) , 
        "T/K"   : np.zeros(4)}
)
states = states.set_index("ID") 
states.head(5)

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0


In [3]:
# Create pandas dataframe for the thermodynamic processes to be filled. Values initialized with 0.
procs = pd.DataFrame(
    {
        "ID" : ["ab","bc","cd","da"],
        "transition" : ["A->B", "B->C", "C->D", "D->A"],
        "type" : ["iso...", "iso...", "iso...", "iso..."], 
        "DU/J" : np.zeros(4) , 
        "Q/J"  : np.zeros(4) , 
        "W/J"  : np.zeros(4) , 
        "DT/K" : np.zeros(4)}
)
procs = procs.set_index("ID") 
procs.head(5)

,transition,type,DU/J,Q/J,W/J,DT/K
ID,,,,,,
ab,A->B,iso...,0.0,0.0,0.0,0.0
bc,B->C,iso...,0.0,0.0,0.0,0.0
cd,C->D,iso...,0.0,0.0,0.0,0.0
da,D->A,iso...,0.0,0.0,0.0,0.0


### Mini-Tutorial: How to Access the Data Frames

The following code cells show examples of getting (reading of retrieving) and setting (writing or assigning) data from and to the data frames. This should help you to solve the tasks appropriately. 

In [4]:
# get values row wise (row = state)
states.loc["A"]

n/mol    0.0
p/Pa     0.0
V/m3     0.0
T/K      0.0
Name: A, dtype: float64

In [5]:
# get values column wise (column = variable)
procs["DU/J"]

ID
ab    0.0
bc    0.0
cd    0.0
da    0.0
Name: DU/J, dtype: float64

In [6]:
# get single value, extract series "DU/J" (column) first, then extract "bc" from the series.
procs["DU/J"]["bc"]

0.0

In [7]:
# get single value, row first (.loc)
states.loc["B", "T/K"] # indexes: row first, column second

0.0

In [8]:
# get single value, named row index first (.loc)
procs.loc["cd", "transition"] # indexes: row first, column second

'C->D'

In [9]:
# get single value, using integer indexing  (.iloc), first index is row number, second index is column number
procs.iloc[2, 0] # indexes: row first, column second

'C->D'

In [10]:
# set (assign) a single value
states.loc["A", "p/Pa"] = -888
states

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,-888.0,0.0,0.0
B,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0


In [11]:
# set (assign) a set of values for a given list of columns
states.loc["B", ["T/K","V/m3"]] = [-888, -777]
states

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,-888.0,0.0,0.0
B,0.0,0.0,-777.0,-888.0
C,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0


In [12]:
# set (assign) a set of values for a full row at once
states.loc["C"] = [-999, -999, -999, -999]
states

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,-888.0,0.0,0.0
B,0.0,0.0,-777.0,-888.0
C,-999.0,-999.0,-999.0,-999.0
D,0.0,0.0,0.0,0.0


In [13]:
# set all values of the whole data frame
states.loc[:,:] = -111.1
states

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,-111.1,-111.1,-111.1,-111.1
B,-111.1,-111.1,-111.1,-111.1
C,-111.1,-111.1,-111.1,-111.1
D,-111.1,-111.1,-111.1,-111.1


In [14]:
# setting (assigning) a set of values at once
states.loc["A"] = [0, 0, 0, 0]
states

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,0.0,0.0,0.0
B,-111.1,-111.1,-111.1,-111.1
C,-111.1,-111.1,-111.1,-111.1
D,-111.1,-111.1,-111.1,-111.1


In [15]:
# set all values of the whole data frame
states.loc[:,:] = 0.0
states

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0


In [16]:
# set a set of values in data frame procs
procs.loc["ab",["type","DU/J","Q/J","W/J","DT/K"]] = ["I don't know ab", -111.1, +222.2, -333.3, +444.4]
procs.head(5)

,transition,type,DU/J,Q/J,W/J,DT/K
ID,,,,,,
ab,A->B,I don't know ab,-111.1,222.2,-333.3,444.4
bc,B->C,iso...,0.0,0.0,0.0,0.0
cd,C->D,iso...,0.0,0.0,0.0,0.0
da,D->A,iso...,0.0,0.0,0.0,0.0


In [17]:
# set a set of values in data frame procs using .iloc (not .loc) for integer indexing
procs.iloc[2,1:4] = ["I don't know cd", +555.5, -666.6]
procs.head(5)

,transition,type,DU/J,Q/J,W/J,DT/K
ID,,,,,,
ab,A->B,I don't know ab,-111.1,222.2,-333.3,444.4
bc,B->C,iso...,0.0,0.0,0.0,0.0
cd,C->D,I don't know cd,555.5,-666.6,0.0,0.0
da,D->A,iso...,0.0,0.0,0.0,0.0


In [18]:
# reset columns of data frame procs
procs.loc[:,["type","DU/J","Q/J","W/J","DT/K"]] = 0.0
procs.head(5)

,transition,type,DU/J,Q/J,W/J,DT/K
ID,,,,,,
ab,A->B,0.0,0.0,0.0,0.0,0.0
bc,B->C,0.0,0.0,0.0,0.0,0.0
cd,C->D,0.0,0.0,0.0,0.0,0.0
da,D->A,0.0,0.0,0.0,0.0,0.0


### Initial Values, Design Values of the Engine

The working piston of the particular Stirling engine under investigation operates between the following max and min volumina:

In [19]:
Vmin_cc = 360 # cc
Vmax_cc = 860 # cc

Use SI units!

In [20]:
Vmin = Vmin_cc*1e-6 # SI units: m^3
Vmax = Vmax_cc*1e-6 # SI units: m^3

The hot and cold temperatures of the working gas (in contact with hot and cold reservoir) are:

In [21]:
Th = 850 # K
Tc = 400 # K

## 1. State A (Initial State)

Start your analysis at State A! It is the **lower left corner** of the p-V-diagram (minimum volume and temperature). Since it is a closed thermodynamic cycle the states are passed cyclicly again and again.<br>The sequence of states looks like  A $\longrightarrow$ B $\longrightarrow$ C $\longrightarrow$ D $\longrightarrow$ A $\longrightarrow$ B $\longrightarrow$ C $\longrightarrow$ D $\longrightarrow$ $\ldots$ ,<br>so A follows after D. You could start the analysis of one cycle at any state but for the sake of comparability with other students' results start at A.


In [22]:
# STATE A (lower left corner in p-V-diagram)
pA = 320e3 # PA <- GIVEN by the design of the engine!
VA = Vmin  # left volume

**Q 1.1:** What is the temperature of State A? 

In [23]:
# Code

# TA = ...

# print(...)

### Amount of Gas in Moles

**Q 1.2:** Use the **ideal gas law** to determine $n$, the number of moles, from state A.

In [24]:
# Code

# n = ...

# print(f"The amount of working gas in the Stirling engine is {n = :.3e} mol")

 **Q 1.3:** Does the amount of gas (moles) change during the cycle (at the different states)? 

**Answer:** ...

### Heat Capacities

**Q 1.4:** The working gas is a diatomic gas. How many degrees of freedom $f$ does the gas have? Explain!

In [25]:
# Code

# f = ...

# print (f"{f = }")

**Answer:** The diatomic gas has ... degrees of freedom, because ... 

**Q 1.5:** What are the **molar heat capacities** (per mol) of the gas at constant volume ($c_v$) and constant pressure ($c_p$) for this $f$, the number of degrees of freedom? Variable names with lower case $c$.

In [26]:
# Code

# cv = 
# cp = 

# print(f"{cv = ...
# print ...

**Q 1.6:** What are the **total heat capacities** of the gas at constant volume ($C_v$) and constant pressure ($C_p$) in $\mathrm{J/K}$? Variable names with upper case $C$.

In [27]:
# Code

# Cv = 
# Cp = 

# print(f"{Cv = :.3f} J/K")
# print ...

### Filling in the Initial Values for State A

In [28]:
states.head()

,n/mol,p/Pa,V/m3,T/K
ID,,,,
A,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0


**Q 1.7** Fill in the values for State A.

In [29]:
# Code

# states.loc["A",["n/mol", "p/Pa","V/m3", "T/K"]] = [n, pA, VA, TA]

# states.head(5)

## 2. State B

**Q 2.1:** What is the pressure $p_B$ of State B? Calculate it. 

In [30]:
# Code

# pB = ...

# print(f"{pB = :.2f} Pa")

**Q 2.2:** Determine the remaining values and fill them in.

In [31]:
# Code

# ...

# states.loc["B"] = [n, pB, VB, TB]

# states.head(5)

## 3. State C

**Q 3.1** Determine all variables of State C and fill it in data frame `states`.

In [32]:
# Code

# pC =
# TC = 
# VC = 

# states.loc["C"] =

# states.head(5)

## 4. State D

**Q 4.1** Determine all variables of State D and fill it in data frame `states`.

In [33]:
# Code

# ...

# states.loc[...

# states.head(5)

## 5. Process _ab_: State Transition A $\rightarrow$ B

In [34]:
procs.head(5)

,transition,type,DU/J,Q/J,W/J,DT/K
ID,,,,,,
ab,A->B,0.0,0.0,0.0,0.0,0.0
bc,B->C,0.0,0.0,0.0,0.0,0.0
cd,C->D,0.0,0.0,0.0,0.0,0.0
da,D->A,0.0,0.0,0.0,0.0,0.0


**Q 5.1:** Which type to state transition happens between A and B? 

In [35]:
# Code 

# type_ab = "iso..."

# print ...

**Q 5.2:** What is the temperature difference $\Delta T_\mathrm{ab}$ between initial state A and final state B? Is the temperature increasing or decreasing? Consider the sign.

In [36]:
# Code

# DTab = ...

# print ...

**Q 5.3:** What is the change of internal energy $\Delta U_\mathrm{ab}$? 

In [37]:
# Code

# DUab = ...

# print ...

**Q 5.4:** How much work $W_\mathrm{ab}$ is done by the system during this process?

In [38]:
# Code

# Wab = ...

# procs.head(5)

**Q 5.5:** How much heat $Q_\mathrm{ab}$ is transferred during this process?

In [39]:
# Code

# Qab = ...

# procs.head(5)

**Q 5.6:** Where does $Q_\mathrm{ab}$ come from? Does it go into the system (part of $Q_\mathrm{in}$) or does it come from the system (part of $Q_\mathrm{out}$)?

**Answer:** ...

**Q 5.7:** Check your results by means of 1st law: $\Delta U = Q + W$.

In [40]:
# Code:

# ...

# print ...

**Q 5.8:** Fill in the values.

In [41]:
# Code

# procs.loc["ab",["type","DU/J","Q/J","W/J","DT/K"]] = [type_ab, DUab, Qab, Wab, DTab]

# alterative: procs.iloc[0,1:] = [type_ab, DUab, Qab, Wab, DTab]

# procs.head(5)

In [42]:
# CHECK A->B
# procs.loc["ab"]

## 6. Process _bc_: State Transition B $\rightarrow$ C

**Q 6.1:** What is the type of the state transition (name of thermodynamic process)?

In [43]:
# Code

# type_bc = ...

# print ...

**Q 6.2:** What is the temperature change and the resulting change of internal energy?

In [44]:
# Code

# DTbc = 
# DUbc =

# print ...

**Q 6.3:** Is the heat $Q_\mathrm{bc}$ going into the system (positive value) or coming from the system (negative value)? Is the work  $W_\mathrm{bc}$ done on the system (positive work) or by the system (negative)?

**Answer:** ...

**Q 6.4:** How much heat and work are transferred? Consider the right signs (positive/negative). Fill all values in.

In [45]:
# Code

# Wbc =
# Qbc = 

# procs.loc["bc",["type","DU/J","Q/J","W/J","DT/K"]] = ...

# procs.head()

**Q 6.5:** Check your results by means of 1st law: $\Delta U = Q + W$.

In [46]:
# Code:

# ...

# print ...

## 7. Process _cd_: State Transition C $\rightarrow$ D

**Q 7.1:** Perform all definitions and calculations accordingly. Fill in the values. 

In [47]:
# Code:

# procs.loc["cd",["type","DU/J","Q/J","W/J","DT/K"]] = [type_cd, DUcd, Qcd, Wcd, DTcd]

# procs.head(5)

## 8. Process _da_: State Transition D $\rightarrow$ A

**Q 8.1:** Perform all definitions and calculations accordingly. Fill in the values. 

In [48]:
# Code:

# procs.loc["da", ...] = ...

# procs.head(5)

## 9. Analysis of this Ideal Stirling Cycle

**Q 9.1:** The Stirling cycle consists of four processes of two different kinds. What are they?

**Answer:** 2 x iso... and 2 x iso...

**Q 9.2:** According to your data frame `procs`: During which processes does heat flow into the system ($Q$ positive)? Name the sum of it $Q_\mathrm{in}$.

In [49]:
# Code

# Qin = ... + ...

# print ...

**Q 9.3:** According to your data frame `procs`: During which processes does heat flow out of the system ($Q$ negative)? Name the sum of it $Q_\mathrm{out}$. $Q_\mathrm{out}$ is negative. 

In [50]:
# Code

# Qout = ... + ...

# print ...

**Q 9.4:** Which of the two processes are associated to work $W$? Explain whether the work is done by the system or done on the system! Consider the sign convention for work defined earlier.

**Answer:** ...

**Q 9.5:** The sum of the work during all processes of the full cycle is the net work $W_\mathrm{net}$. Is it positive or negative? What does the sign mean?

In [51]:
# Code

# Wnet = ... 

# print ...

**Answer:** ...

**Q 9.6:** How much energy do you have to invest for a full cycle? This is what you have to pay for, e.g. when you use a fossil fuel heat source. How much heat are you wasting, losing? What is the usable work you gain from the system?

**Answer:** ...

**Q 9.7:** What is the efficiency $\eta$ of the engine? It is the ratio of usable work per invested energy input.

In [52]:
# Code

# eta = ... 

# print ...

**Q 9.8:** Is $\eta$ good or bad? Compare it to the theoretically maximum efficiency, the Carnot efficiency, for the given hot and cold reservoir temperatures.

In [53]:
# Code

# eta = ... 

# print ...

**Answer:** ...

**Q: 9.9:** Assume the engine runs at **1500 rpm**. How much mechanical power $P_\mathrm{mech}$ in Watts does it generate?

In [54]:
# Code

# Pmech  = ...

# print ...

**Q 9.10:** How much thermal input power $P_\mathrm{in}$ from the heat source is needed to provide the mechanical power $P_\mathrm{mech}$? 
<br>Which irradiance component is the main source of solar radiant power $P_\mathrm{in}$ collected by the mirror mounted to the 2-axis solar tracker? 

In [55]:
# Code

# Pin = ...

# print ...

**Answer:** ...

## 10. Plotting the p-V-Diagram


<img src="./images/p-V-Diagram.png" width="400">

_Fig.2: p-V diagram template of the Stirling engine under study to be prepared by you._

**Q 10.1:** Use matplotlib to plot the p-V Diagram for the Stirling engine under study! Produce your own diagram similar to the one above!
<br>Try to get as close to the template as possible! The complex diagram has a lot of features.<br>
**Do not just copy** the values and the diagram! The diagram has to be **BASED ON YOUR OWN CALCULATIONS** from the Stirling engine analysis above!<br>
You **must plot the segments** $B\rightarrow C$ and $D\rightarrow A$ corrently **curved!** They **are not straight lines!** They are representing a particular thermodynamic process! 
<br>Replace "iso..." in the diagram legend with the **correct names** of the processes. 

In [56]:
# Code

# V = np.linspace(Vmin,Vmax,100)
# calculate the correct curved segments ...

# fig1, ax1 = plt.subplots(figsize=(5,5), dpi=150, subplot_kw={'aspect': 'equal'})
# fill_color = (0.8,0.9,0.8)

# ax1.plot(VA*1e6,pA/1000,"ko")
# ...
# ax1.annotate("$A$", (320,310))
# ...
# ax1.plot([VA*1e6,VB*1e6],[pA/1000,pB/1000],label= r"$A\rightarrow B$: iso...")
# ...
# ax1.fill_between(...)
# 
# ax1.set_ylabel(r"Pressure $p$ / $\mathrm{kPa}$")
# ...

# plt.show()

## 11. The Geographic Location and its Irradiance

The geographical location determines the long term characteristics of the irradiance. The irradiance concentrated by the parabolic reflector provides the input power for your Stirling engine. Both goes together.

**Your tasks:**

1. **Select a location** in your home country where you would like to install your Dish-Stirling sytem.
1. Determine the **average yearly irradiance components** (direct, diffuse, global) from a **typical meteorological year (TMY)** data **for your location.**
1. **Dimension the dish size** of your system to collect enough radiant power to feed your engine.

Use the [**PVGIS online tools**](https://re.jrc.ec.europa.eu/pvg_tools/en/) of the _Joint Research Center (JRC)_ of the European Commission to select a location and to download the TMY data for your selected location!

**Remark: This task is a vast simplification!** A more precise performance analysis would require to use time series irradiance data e.g. in hourly resolution and feed this time varying concentrated solar power into your Stirling engine. Under some  low irradiance condition the engine would not even start to drive the generator mounted to it. The whole system is not linear! 
<br>
Anyway, to keep the analysis simple we use the average solar radiation calculated from the TMY data. We use just one "representative" average irradiance value as mean source power. This mean irradiance (measured in $\mathrm(Wm^{-2})$ has to be concentrated (collected) by the dish of mirror area $A$ (measured in $m^2$). The size of the dish must be so large, that the average concentrated power is as large as the necessary input power $P_\mathrm{in}$ determined before. Again, this analysis just yields rough estimates. It is not precise enough for expensive real world installations!   






<img src="./images/PVGIS-Tools.png" width="800">

_Fig.3: The PVGIS online tools of the _Joint Research Center (JRC)_ of the European Commission. In this example a location near the city of Xanten is selected._

**Q 11.1:** Open the [**PVGIS online tools**](https://re.jrc.ec.europa.eu/pvg_tools/en/) and **select a location** in your home country. Accept the default settings of the tools. Go to the tab `TMY`, **download the TMY data in CSV format** and move it to the local subfolder `./data/`. In the folder there is an already downloaded file of the location Xanten named `tmy_51.649_6.461_2005_2020.csv`. The name indicates that the data belongs to the location at the coordinates 51.649° north latitude and 6.461° east longitude.

You have to use **your own location** for the analysis! The file you download **must be different!** You have to adapt the following code to the file name of your downloaded TMY data.

In [57]:
# List the content of the data directory.
import pathlib
p = pathlib.Path('.')
for path in p.glob('./data/tmy*.csv'):
    print(path)

data\tmy_51.649_6.461_2005_2020.csv


**Change the following path-filename to your own TMY data file for your selected location!**

In [58]:
pfname = "./data/tmy_51.649_6.461_2005_2020.csv"

**Q 11.2:** Before you continue **analyse the CSV file!** Use the file `tmy_51.649_6.461_2005_2020.csv` as an example. Use a text editor (e.g. `notepad++` if you are using Windows) and open the file. **How is it structured?** What do the text blocks contain? Complete the table below. Describe in short keywords.

**Answer:** (complete the table)

| **rows** | **content** |
| --- | --- |
| 1 - 3 | |
| 4 | column names for the text block with the typical (most representative) months and the years from which the data originate |
| 5 - 16 | |
| 17 | |
| 18 - 761 | |
| 721 - 1433 | |
| 8778 | empty line |
| 8779 - 8787 | |
| 8789 | |

**Q 11.3:** Which year provides the most representative data on solar radiation in June? Which row number range contains the June data?

**Answer:** ...

**Q 11.4:** What do the columns `G(h)`, `Gb(n)`, and `Gd(h)` contain?

**Answer:**

**Q 11.5:** Read the TMY radiation data (CSV file) into a pandas dataframe. **Now you your own downloaded TMY data!** You have to crop the corresponding data block from the CSV file. The easiest way is to skip the rows above and below the relevant range of rows. To make yuor life easier I provide some rudimentary code which runs for the Xanten example. Maybe you have to adapt the row ranges to be excluded.

In [59]:
idx_exclude = list(range(0,16)) + list(range(8777,8790))
tmy = pd.read_csv(pfname, skiprows = idx_exclude)

In [60]:
tmy.head()

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP
0,20180101:0000,5.21,86.50,0.0,-0.0,0.0,287.57,5.03,230.0,99222.0
1,20180101:0100,5.55,84.87,0.0,-0.0,0.0,286.36,5.48,227.0,99202.0
2,20180101:0200,5.88,83.24,0.0,-0.0,0.0,285.15,5.94,226.0,99252.0
3,20180101:0300,6.21,81.61,0.0,-0.0,0.0,283.93,6.39,227.0,99282.0
4,20180101:0400,6.55,79.98,0.0,-0.0,0.0,282.72,6.84,234.0,99332.0


In [61]:
tmy.tail()

,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP
8755,20141231:1900,3.54,94.65,0.0,-0.0,0.0,293.65,2.77,234.0,102879.0
8756,20141231:2000,3.88,93.02,0.0,-0.0,0.0,292.43,3.22,232.0,102899.0
8757,20141231:2100,4.21,91.39,0.0,-0.0,0.0,291.22,3.67,229.0,102909.0
8758,20141231:2200,4.54,89.76,0.0,-0.0,0.0,290.00,4.13,232.0,102929.0
8759,20141231:2300,4.88,88.13,0.0,-0.0,0.0,288.79,4.58,232.0,102948.0


**Q 11.6:** Calculate the mean irradiance $G_\mathrm{mean}$ of the radiation component relevant for the Dish-Stirling system with solar tracker. Utilize the vectorized functions of numpy or pandas, respectively. What is the unit of this mean? Print the result appropriately, i.e. formatted including unit.

In [62]:
# Code

# Gmean = 

# print(f"...")

**Q 11.7:** What area $A$ and diameter $d$ must the circular solar dish have to concentrate enough thermal input power $P_\mathrm{in}$ for the engine?
<br>If you were not able to calculate $P_\mathrm{in}$ you could use the design value $P_\mathrm{in} = 13\,\mathrm{kW}$

In [63]:
# Code

# A = 
# d = 

# print(f"...")
# ...

**Q 11.8:** The size is probably huge! Why is this the case? **Is this realistic?** Which or our system usage assumptions are questionable? How should we change the system usage assumptions to achieve more realistic results? **Discuss!** 

**Answer:**

**Q 11.9:** Fig.1 of this Jupyter notebook shows a real (old) Dish-Stirling experiment. In the local subfolder `./literature/` you find an article (pdf) with detailed performance analyses. The authors state that the dish is oversized. What is the consequence? Which technical measures had to be taken to handle the situation? 

**Answer:**

**Q 11.10:** The performance analysis is partly based on simulations. The simulation is based on an "insolation" (what I called irradiance) of $906\,\mathrm{Wm^{-2}}$. It seems that the diameter (area) of the dish is not given, but you can estimate it indirectly from Table 3 by means of _"Power from dish"_. Which diameter $d_\mathrm{ED}$ of the dish (subscript ED stands for EuroDish) follows from these quantities?


In [64]:
# Code

**Q 11.11:** This collected power is concentrated to the absorber. This heat transfer is associated with several losses such that the _"Thermal power into engine"_ (Table 3) is reduced. The engine transforms this input power to usable mechanical output power at the rotating shaft named _"Stirling shaft power"_ in Table 3. What is the resulting "mechanical" efficiency of the Stirling engine converting heat to work?

In [65]:
# Code

## 12. Feedback

Would you please give me a personal feedback? How do you like this comprehensive task? 

**Answer:** I think that this task is ...